# A1

## A.1.3

In [1]:
import torch
torch.__version__ # 2.2.2

'2.2.2'

In [2]:
torch.cuda.is_available() # False

False

In [3]:
print(torch.backends.mps.is_available()) # False

False


### Exercise A.1
Finished installing and setting up PyTorch on my computer.

### Exercise A.2
Finished running the supplementary code and everything outputed correctly. My environment is set up correctly.

# A2

## A.2.1

In [4]:
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2], [3, 4]])
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])


## A.2.2

In [5]:
tensor1d = torch.tensor([1, 2, 3])
print(tensor1d.dtype)

torch.int64


In [6]:
floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

torch.float32


In [7]:
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

torch.float32


## A.2.3

In [8]:
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(tensor2d)

tensor([[1, 2, 3],
        [4, 5, 6]])


In [9]:
print(tensor2d.shape)
# 2 rows and 3 columns

torch.Size([2, 3])


In [10]:
print(tensor2d.reshape(3, 2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [11]:
print(tensor2d.view(3, 2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [12]:
print(tensor2d.T)

tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [13]:
print(tensor2d.matmul(tensor2d.T))

tensor([[14, 32],
        [32, 77]])


In [14]:
print(tensor2d @ tensor2d.T)

tensor([[14, 32],
        [32, 77]])


# A.3

In [15]:
import torch.nn.functional as F
y = torch.tensor([1.0])             # True label
x1 = torch.tensor([1.1])            # Input feature
w1 = torch.tensor([2.2])            # Weight parameter
b = torch.tensor([0.0])             # Bias unit
z = x1 * w1 + b                     # Net input
a = torch.sigmoid(z)                # Activation and output
loss = F.binary_cross_entropy(a, y)
print(f"The loss is {loss}.")

The loss is 0.0851878821849823.


# A.4

In [16]:
# Computing gradients via autograd
from torch.autograd import grad
y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

In [17]:
# Using grad function manually
z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)
grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [18]:
# Automated process - PyTorch compute the gradients of all the leaf nodes in this graph.
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


# A.5

In [19]:
# A multilayer perceptron with two hidden layers
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30), # 1st hidden layer
            torch.nn.ReLU(),
            torch.nn.Linear(30, 20), # 2nd hidden layer
            torch.nn.ReLU(),
            torch.nn.Linear(20, num_outputs) # output layer
        )
    
    def forward(self, x):
        return self.layers(x)

In [20]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [21]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable model parameters: {num_params}")

Total number of trainable model parameters: 2213


In [22]:
print(model.layers[0].weight)
# Random output each time

Parameter containing:
tensor([[-0.0211,  0.1404, -0.1336,  ...,  0.0870,  0.0818, -0.0192],
        [ 0.1192,  0.0735,  0.0177,  ...,  0.1182,  0.0969,  0.0113],
        [ 0.1011,  0.0720,  0.1256,  ..., -0.0556,  0.0134, -0.1223],
        ...,
        [-0.0790,  0.1107,  0.1018,  ..., -0.0748,  0.0930, -0.0491],
        [-0.0217,  0.0746, -0.0441,  ...,  0.0632,  0.0387, -0.0345],
        [ 0.0215,  0.1221, -0.0632,  ...,  0.0280,  0.1289,  0.0432]],
       requires_grad=True)


In [23]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [24]:
print(model.layers[0].bias)

Parameter containing:
tensor([ 0.1064, -0.0191, -0.0662,  0.0002,  0.0050,  0.1071,  0.0682,  0.0687,
         0.0077, -0.1255,  0.0132, -0.1355,  0.1031, -0.0571, -0.1064, -0.0658,
         0.0089,  0.0789, -0.1357, -0.0474,  0.0275,  0.0517, -0.0323,  0.0637,
        -0.0401, -0.0957, -0.1290, -0.0457, -0.0345,  0.0046],
       requires_grad=True)


In [25]:
print(model.layers[0].bias.shape)

torch.Size([30])


In [26]:
# Make the random number initialization reproducible
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [27]:
# A forward pass
torch.manual_seed(123)
X = torch.rand((1, 50))
output = model(X)
print(output)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [28]:
# For prediction after training
with torch.no_grad():
    output = model(X)
print(output)

tensor([[-0.1262,  0.1080, -0.1792]])


In [29]:
# Compute class-membership probabilities
with torch.no_grad():
    output = torch.softmax(model(X), dim=1)
print(output)

tensor([[0.3113, 0.3934, 0.2952]])


# A.6

In [30]:
# Creating a small toy dataset
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0,0,0,1,1])
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0,1])

In [31]:
# Defining a custom Dataset class
from torch.utils.data import Dataset
class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        this_x = self.features[index]
        this_y = self.labels[index]
        return this_x, this_y
    
    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)
print(len(train_ds))

5


In [32]:
# Instantiating data loaders
from torch.utils.data import DataLoader
torch.manual_seed(123)

train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0 # The number of background processes
)

test_loader = DataLoader(
    dataset = test_ds,
    batch_size = 2,
    shuffle = False,
    num_workers = 0
)

In [33]:
for idx, (x, y) in enumerate(train_loader):
    print(f'Batch {idx + 1}:', x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [34]:
# Drop the last batch
train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0, # The number of background processes
    drop_last = True
)

In [35]:
for idx, (x, y) in enumerate(train_loader):
    print(f'Batch {idx + 1}:', x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


# A.7

In [36]:
# Neural network training in PyTorch
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.5)
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Logging
        print(f"Epoch: {epoch+1:d}/{num_epochs:d} | Batch {batch_idx:d}/{len(train_loader):d} | Train Loss: {loss:.5f}")
    model.eval()

Epoch: 1/3 | Batch 0/2 | Train Loss: 0.74874
Epoch: 1/3 | Batch 1/2 | Train Loss: 0.64503
Epoch: 2/3 | Batch 0/2 | Train Loss: 0.44226
Epoch: 2/3 | Batch 1/2 | Train Loss: 0.12562
Epoch: 3/3 | Batch 0/2 | Train Loss: 0.02691
Epoch: 3/3 | Batch 1/2 | Train Loss: 0.00433


### Exercise A.3
- Input layer: 2 x 30 + 30 = 90
- Second layer: 30 x 20 + 20 = 620
- Output layer: 20 x 2 + 2 = 42
- Total: 90 + 620 + 42 = 752 parameters.

In [37]:
# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [38]:
# Obtain the class membership probabilities
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [39]:
# Convert class membership probabilities into class label predictions
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [40]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [41]:
predictions == y_train

tensor([True, True, True, True, True])

In [42]:
torch.sum(predictions == y_train)

tensor(5)

In [43]:
# Generalize the computation of the prediction accuracy
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total = 0
    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total += len(compare)
    return (correct / total).item()

In [44]:
print(compute_accuracy(model, train_loader))

1.0


In [45]:
print(compute_accuracy(model, test_loader))

1.0


# A.8

In [46]:
# Save models in PyTorch
torch.save(model.state_dict(), "model.pth")

In [47]:
# Retore the model from disk
model = NeuralNetwork(2, 2)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>